In [1]:
from statsmodels.stats.weightstats import ztest

import pandas as pd
import numpy as np

from tabulate import tabulate
from IPython.display import HTML, display

from datetime import datetime

### Getting 10 years of daily RUB/USD rates from CBR

In [2]:
data = pd.read_csv('FX_weekday_trades.csv', dtype={'Rate': float}, parse_dates=['Date'])
data['Weekday'] = data['Date'].dt.weekday
data['Year_num'] = data['Date'].dt.year
data['Week_num'] = data['Date'].dt.week

### Assigning weekdays to dates 

In [3]:
Weekday_map = dict(zip(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']))
data['Weekday'] = data['Weekday'].replace(Weekday_map)

### Removing rare weekdays with insufficient amount of observations

In [4]:
min_number = 100
Weekdays = []

for weekday in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    if (len(data[data['Weekday'] == weekday]) >= min_number):
        Weekdays.append(weekday)
        
data = data[data['Weekday'].isin(Weekdays)]

### Reindexing dataset from exact dates to year-weeks

In [5]:
data_transformed = pd.pivot_table(data, values='Rate', index=['Year_num', 'Week_num'], columns='Weekday')

### Conducting z-test of positive average return for each weekday pair

In [10]:
Results_table = [['Buy weekday', 'Sell weekday', 'Num of obs', 'Avg weekly return, %', 't-stat', 'p-val']]

for buy_day in Weekdays:
    for sell_day in (set(Weekdays)-set([buy_day])):
        
        # Matching weekdays with corresponding USD/RUB rates
        if (Weekdays.index(sell_day) > Weekdays.index(buy_day)):
            buy_rates = data_transformed[buy_day]
            sell_rates = data_transformed[sell_day]
        else:
            buy_rates = data_transformed[buy_day]
            sell_rates = data_transformed[sell_day].shift(-1)
        
        # Calculating weekly returns of the strategy
        Strategy_Returns = np.log(sell_rates) - np.log(buy_rates)
        Strategy_Returns.dropna(inplace=True)
        
        # Conducting z-test
        mean_val = np.round(100*np.mean(Strategy_Returns), 2)
        t_stat, p_val = np.round(ztest(x1=Strategy_Returns, value=0, alternative='larger'), 4)
        
        # Saving results
        Results_table.append([buy_day, sell_day, len(Strategy_Returns), mean_val, t_stat, p_val])

### Presenting z-test results

In [11]:
display(HTML(tabulate(Results_table, tablefmt='html')))

Buy weekday,Sell weekday,Num of obs,"Avg weekly return, %",t-stat,p-val
Tuesday,Thursday,463,0.08,1.1973,0.1156
Tuesday,Saturday,448,0.02,0.2399,0.4052
Tuesday,Friday,457,-0.02,-0.2727,0.6075
Tuesday,Wednesday,472,-0.01,-0.1556,0.5618
Wednesday,Tuesday,457,0.27,1.7234,0.0424
Wednesday,Saturday,474,-0.03,-0.3561,0.6391
Wednesday,Thursday,489,0.02,0.397,0.3457
Wednesday,Friday,483,-0.08,-1.298,0.9029
Thursday,Tuesday,464,0.27,1.7182,0.0429
Thursday,Saturday,482,-0.04,-0.6263,0.7344


### Conclusion: the results of z-test suggest that there is only one weekday pair which yields positive average return on the 1% confidence level. The corresponding strategy is to buy Saturday's USD/RUB rate and sell next Wednesday's USD/RUB rate. Average weekly return of this strategy is 0.23%.